Code de generation des données:

In [3]:
import os
import csv
import numpy as np
from string import ascii_lowercase

n, p, m = 10, 7, 11
# n est le nombre d'individus par lettre
# p est le nombre de snapshot par  (nombre de capture dans un delta t)
# m est le nombre de variables qu'on a (5 flex + 6 accelérations données par le gyroscope:: Les accelerations de rotation et translation)

# Create base directory for dataset
base_dir = "data_generated"
os.makedirs(base_dir, exist_ok=True)

for i in range(26):
    # Create directory for each letter
    letter = ascii_lowercase[i]
    letter_dir = os.path.join(base_dir, letter)
    os.makedirs(letter_dir, exist_ok=True)
    
    for k in range(n):
        # Create CSV file for each repetition
        file_path = os.path.join(letter_dir, f"sample_{k}.csv")
        
        with open(file_path, 'w', newline='') as f:
            writer = csv.writer(f)
            
            for j in range(p):
                # Generate m random numbers with different distributions
                row = []
                
                # Mix of normal, uniform, and other distributions for variety
                row.extend(np.random.normal(450, 100, size=4).astype(int).clip(100, 800))
                row.extend(np.random.uniform(100, 800, size=4).astype(int))
                row.extend(np.random.exponential(scale=200, size=3).astype(int).clip(100, 800))
                
                writer.writerow(row)

## Lecture et re-Organisation des données recoltées

Il s'agit de lire et restocker
un dossier par position
un fichier par lettre


In [13]:
import os
import csv
import pandas as pd
from string import ascii_lowercase

base_dir = "data_generated"
new_base_dir = "data"

# Create new directory structure
os.makedirs(new_base_dir, exist_ok=True)
for pos in range(p):
    os.makedirs(os.path.join(new_base_dir, f"position_{pos}"), exist_ok=True)

# Reorganize by position
for pos in range(p):

    # Process each letter
    for letter in ascii_lowercase:

        # Read all files for this letter
        letter_data = []
        letter_dir = os.path.join(base_dir, letter)
        for k in range(n):
            file_path = os.path.join(letter_dir, f"sample_{k}.csv")
            with open(file_path, 'r') as f:
                reader = csv.reader(f)
                l = 0
                for row in reader:
                    if l == pos:
                        letter_data.extend([row])
                        break
                    l += 1

        position_file = os.path.join(
            new_base_dir, f"position_{pos}", f"{letter}.csv")

        # Extract position data and save
        with open(position_file, 'w', newline='') as f:
            writer = csv.writer(f)
            for row in letter_data:
                writer.writerow(row)

# Analyse
Il s'agit ici d'ecrire un script qui permettra d'avoir une representation unique pour chaque lettre
On commence par charger les données,

In [97]:
from string import ascii_lowercase

# initialisation
result = {}
data = {}

for letter in ascii_lowercase:
    result[letter] = [0 for i in range(m)]
    data[letter] = np.array([[[0 for k in range(m)]for j in range(n)] for i in range(p)])

# chargement des données
for letter in ascii_lowercase:
    for k in range(p):
        sample_path = os.path.join(new_base_dir, f"position_{k}", f"{letter}.csv")
        with open(sample_path, mode="r") as sample_file:
            reader = csv.reader(sample_file)
            data[letter][k] =np.array([np.array(row) for row in reader])


### Version 1: Representation des lettre par un vecteur de dimension m par double barycentre
Pour chaque lettre, on va faire la moyenne des individus, à chaque position, puis la moyenne pondérée des position 


In [258]:
from scipy.stats import norm

# clacul de la population moyenne
population_mean = dict()

for letter in ascii_lowercase:
    population_mean[letter] = np.mean(data[letter], axis=1)

# calcul de l'etat moyen pour chaque lettre


def compute_mean_state(mean_population=population_mean, use_normal_law=True, a=-3, b=3):
    mean_states = dict()
    if not use_normal_law:
        for letter in ascii_lowercase:
            mean_states[letter] = np.average(
                mean_population[letter], axis=0)
    if use_normal_law:
        for letter in ascii_lowercase:
            p = mean_population[letter].shape[0]
            weights = np.random.normal(0, 1, p)
            # Générer des points équidistants entre -3 et 3
            x = np.linspace(a, b, p)
            # Densité de probabilité de la loi normale centrée réduite
            weights = norm.pdf(x, loc=0, scale=1)
            # Normaliser les poids pour que leur somme soit égale à 1sum
            weights /= np.sum(weights)

            mean_states[letter] = np.average(
                mean_population[letter], axis=0, weights=weights)

    return mean_states


mean_states = compute_mean_state()

### Version2: Usage complexe

Je vais d'abord ecrire, celui qui va comprendre, comprendra


In [252]:
# calculer les moyenne des individus à chaque position
def compute_mean_positions(data=data, use_normal_law=True, a=-3, b=3):
    mean_positions = dict()
    if not use_normal_law:
        for letter in ascii_lowercase:
            mean_positions[letter] = np.average(
                data[letter], axis=1)
    if use_normal_law:
        for letter in ascii_lowercase:
            p = data[letter].shape[1]
            weights = np.random.normal(0, 1, p)
            # Générer des points équidistants entre -3 et 3
            x = np.linspace(a, b, p)
            # Densité de probabilité de la loi normale centrée réduite
            weights = norm.pdf(x, loc=0, scale=1)
            # Normaliser les poids pour que leur somme soit égale à 1sum
            weights /= np.sum(weights)

            mean_positions[letter] = np.average(
                data[letter], axis=1, weights=weights)

    return mean_positions

mean_positions = compute_mean_positions()

# Prediction
Ici on aura deux algos, un par méthode, les deux ont des entrées similaires mais des sorties differentes.

In [350]:
def key_of_min(d):
  return min(d, key = d.get)

input = np.array([
    np.array([467,452,493,458,771,638,408,229,100,333,201]),
    np.array([415,498,514,559,442,605,763,576,455,443,173]),
    np.array([563,416,443,337,756,283,508,112,100,100,223]),
    np.array([444,321,607,334,685,576,435,647,129,278,203]),
    np.array([513,411,437,402,360,484,259,343,519,100,100]),
    np.array([284,591,535,456,149,176,173,378,206,290,100]),
    np.array([414,434,485,488,523,725,308,669,211,230,100])
])


## Version 1 -- adaptée à la premiere analyse


In [353]:
def predict_v1(input=input, mean_states=mean_states):
    """input est une matrice de tailles p*m"""
    mean_input = np.mean(input, axis=0) # donne un vecteur de taille m
    # calculer la distance de 
    distances = dict()
    for letter in ascii_lowercase:
        distances[letter] = np.linalg.norm(mean_input - mean_states[letter])
    
    # TODO: gerer le cas où le min est atteint en plusieures lettres
    # TODO: Evaluer la qualité de la prediction, grace à l'ecart type. si d>e alors catastrophique, sinon, loi uniforme
    return key_of_min(distances)

predict_v1(input)

'w'

### Verison 2: Liée à la seconde analyse

In [352]:
def predict_v2(input, mean_positions=mean_positions, decision_strategy="vote"):
    distances = list()
    for letter in ascii_lowercase:
        distances.append(np.linalg.norm(input-mean_positions[letter], axis=1))

    distances = np.array(distances)
    predictions = np.argmin(distances, axis=0)
    
    predictions = [ascii_lowercase[i] for i in predictions] #['c', 'a', 'b', 'o', 'f', 'u', 'j']

    # il est temps de decider
    if decision_strategy=="vote":
        # TODO generer le cas où il y a equi presence.
        return max(predictions, key=predictions.count)
predict_v2(input)

'a'

In [357]:
data["a"][:][:][0].shape

(10, 11)